In [197]:
from biotite.structure.io.pdb import PDBFile
from biotite.structure import AtomArray, Atom
import numpy as np
from visualize import backbone_to_pdb
from sidechainnet.utils.sequence import THREE_TO_ONE_LETTER_MAP
from biotite.structure import filter_amino_acids, filter_nucleotides, get_residues, get_residue_count
from biotite.sequence import ProteinSequence

In [321]:
pdb_file = PDBFile.read("data/cath_sanity/1gpwB00.pdb")
struct = pdb_file.get_structure()

/Users/ido/genomator/venv/lib/python3.9/site-packages/biotite/structure/io/pdb/file.py:423: UserWarning: 1615 elements were guessed from atom_name.
  warn("{} elements were guessed from atom_name.".format(rep_num))


In [322]:
struct = pdb_file.get_structure(model=1, extra_fields=["atom_id", "b_factor", "occupancy", "charge"])

In [292]:
dna_indices = [i for i, atom in enumerate(struct) if is_res_dna(atom.res_name)]
res_indices = [i for i, atom in enumerate(struct) if not is_res_dna(atom.res_name)]

In [316]:
dna_array = struct[dna_indices]

In [294]:
dna_seq = "".join([x[1] for x in get_residues(dna_struct)[1]])

In [295]:
atoms_per_nuc = []
curr = []
curr_res_id = dna_struct[0].res_id
for atom in dna_struct:
    if atom.res_id != curr_res_id:
        atoms_per_nuc.append(curr)
        curr = []
        curr_res_id = atom.res_id
    curr.append(atom)

In [296]:
[f"{len(x), dna_seq[i]}" for i, x in enumerate(atoms_per_nuc)]

["(28, 'C')",
 "(32, 'T')",
 "(32, 'A')",
 "(33, 'G')",
 "(30, 'C')",
 "(33, 'G')",
 "(30, 'C')",
 "(33, 'G')",
 "(30, 'C')",
 "(32, 'T')",
 "(32, 'A')",
 "(34, 'G')",
 "(28, 'C')",
 "(32, 'T')",
 "(32, 'A')",
 "(33, 'G')",
 "(30, 'C')",
 "(33, 'G')",
 "(30, 'C')",
 "(33, 'G')",
 "(30, 'C')",
 "(32, 'T')",
 "(32, 'A')"]

In [320]:
dna_array[0]

Atom(np.array([ -5.2  , -16.889,   8.179], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="DC", hetero=False, atom_name="O5'", element="O")

In [317]:
_, nuc_names = get_residues(dna_array)
nuc_names = [n[1] for n in nuc_names]  # in PDB nucleotides start with D
dna_sequence = "".join(nuc_names)

In [304]:
nuc_atom_indices = []
curr_nuc_id = dna_array[0].res_id
num_atoms = len(dna_array)
for i in range(num_atoms):
    if dna_array[i].res_id != curr_nuc_id:
        nuc_atom_indices.append(i)
        curr_nuc_id = dna_array[i].res_id
nuc_atom_indices.append(num_atoms)

# retrieve atoms per nucleotide and masks
dna_atoms = np.zeros((len(dna_sequence), 34, 3))
dna_mask = np.zeros((len(dna_sequence), 34))
prev_idx = 0
for i, idx in enumerate(nuc_atom_indices):
    num_nuc_atoms = idx - prev_idx
    curr_nuc_atoms = dna_array.coord[prev_idx:idx]
    dna_atoms[i][:num_nuc_atoms] = curr_nuc_atoms
    dna_mask[i][:num_nuc_atoms] = 1
    prev_idx = idx


In [314]:
dna_mask[1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.])

In [315]:
dna_atoms[1]

array([[-3.03099990e+00, -1.33459997e+01,  9.81400013e+00],
       [-3.77300000e+00, -1.29759998e+01,  1.10400000e+01],
       [-3.24699998e+00, -1.25839996e+01,  8.56400013e+00],
       [-1.45500004e+00, -1.33219995e+01,  1.01510000e+01],
       [-9.86000001e-01, -1.27489996e+01,  1.13760004e+01],
       [ 1.84000000e-01, -1.18170004e+01,  1.10990000e+01],
       [ 1.03699994e+00, -1.24309998e+01,  1.01129999e+01],
       [-3.19999993e-01, -1.05349998e+01,  1.04750004e+01],
       [-3.26000005e-01, -9.52200031e+00,  1.14809999e+01],
       [ 6.53999984e-01, -1.01560001e+01,  9.39900017e+00],
       [ 1.45700002e+00, -1.14209995e+01,  9.16699982e+00],
       [ 1.25600004e+00, -1.19549999e+01,  7.81599998e+00],
       [ 2.37199998e+00, -1.23100004e+01,  7.08300018e+00],
       [ 3.50999999e+00, -1.22240000e+01,  7.53800011e+00],
       [ 2.12899995e+00, -1.27679996e+01,  5.80000019e+00],
       [ 8.84999990e-01, -1.28959999e+01,  5.20300007e+00],
       [ 7.82000005e-01, -1.33160000e+01

In [204]:
res_ids, res_names = get_residues(struct)

In [199]:
get_residue_count(struct)

150

In [201]:
len(struct)

2834

In [194]:
def is_res_dna(res):
    return len(res) == 2 and res[0] == "D"

In [195]:
res_names = [res[1] for res in res_names if is_res_dna(res)]

In [196]:
res_names

['C',
 'T',
 'A',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'T',
 'A',
 'G',
 'C',
 'T',
 'A',
 'G',
 'C',
 'G',
 'C',
 'G',
 'C',
 'T',
 'A',
 'G']

In [88]:
len(struct[0])

2834

In [79]:
struct[0][-300]

Atom(np.array([-12.104,  -2.779,  -8.42 ], dtype=float32), chain_id="D", res_id=47, ins_code="", res_name="GLU", hetero=False, atom_name="N", element="N")

In [89]:
backbone_indices = []
res_count = 0
curr_res_id = 0
num_backbone_atoms = 4
for i, atom in enumerate(struct[0]):
    if is_res_dna(atom.res_name):
        backbone_indices.append(i)
        res_count = 0
    else:
        # new residue
        if atom.res_id != curr_res_id:
            curr_res_id = atom.res_id
            res_count = 0
        # same residue only backbone atoms
        if res_count < num_backbone_atoms:
            backbone_indices.append(i)
            res_count += 1

In [90]:
backbone_atoms = struct[0][backbone_indices]

In [92]:
pdb_file.set_structure(backbone_atoms)

In [93]:
pdb_file.write("data/1bbx-bb.pdb")

In [127]:
pdb_file = PDBFile.read("data/1bbx-bb.pdb")
struct = pdb_file.get_structure()

In [130]:
struct.coord.shape

(1, 1262, 3)

In [131]:
struct

stack([
	array([
	Atom(np.array([ -5.2  , -16.889,   8.179], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="DC", hetero=False, atom_name="O5'", element="O"),
	Atom(np.array([ -4.015, -17.617,   7.845], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="DC", hetero=False, atom_name="C5'", element="C"),
	Atom(np.array([ -2.787, -17.001,   8.495], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="DC", hetero=False, atom_name="C4'", element="C"),
	Atom(np.array([ -1.627, -17.452,   7.779], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="DC", hetero=False, atom_name="O4'", element="O"),
	Atom(np.array([ -2.813, -15.503,   8.311], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="DC", hetero=False, atom_name="C3'", element="C"),
	Atom(np.array([ -3.306, -14.903,   9.509], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="DC", hetero=False, atom_name="O3'", element="O"),
	Atom(np.array([ -1.392, -15.061,   8.073

In [158]:
crds = [x.coord.tolist() for x in struct[0] if not x.res_name.startswith("D")]
residues = [THREE_TO_ONE_LETTER_MAP[x.res_name] for x in struct[0] if not x.res_name.startswith("D")]
indices = [i for i in range(len(residues)) if i % 4 == 0]
seq = "".join([residues[i] for i in indices])

In [160]:
backbone_to_pdb(crds, seq, "data/test.pdb", 4)

File data/test.pdb has been saved.
